In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, RandomSampler
import tqdm
from BOW_data_format import prepare_data

In [2]:
prepared = prepare_data()
title_data = prepared['title_data']
train_data = prepared['train']
validation_data = prepared['validation']
test_data = prepared['test']
vocab = prepared['vocab']
PAD_ID = prepared['PAD_ID']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
titles = nn.utils.rnn.pad_sequence([torch.LongTensor(title_data[i]) if i in title_data else torch.LongTensor([PAD_ID]) for i in range(max(title_data) + 1)], batch_first=True).to(device).contiguous()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## Create Data Loaders

In [37]:
def bag_of_words_collate(batch):
    batch_arr = np.array(batch).T
    data = titles[batch_arr[0]]
    labels = torch.FloatTensor(batch_arr[1]).to(device)
    return data, labels.squeeze()

In [38]:
BATCH_SIZE = 8
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=RandomSampler(train_data), collate_fn=bag_of_words_collate)
validation_iter = DataLoader(validation_data, sampler=RandomSampler(validation_data), batch_size=1, collate_fn=bag_of_words_collate)
test_iter = DataLoader(test_data, sampler=RandomSampler(test_data), batch_size=1, collate_fn=bag_of_words_collate)

## Make the Model

In [39]:
class BagOfWords(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout=0.1):
        super(BagOfWords, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(embedding_dim, 1)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        out = self.dropout(self.embedding(x))
        out = out.mean(1)
        out = self.activation(self.linear(out)).squeeze()
        return out


## Evaluation

In [40]:
def eval_model(model, data_iter):
    model.eval()
    predictions, all_labels= [], []
    for (data, labels) in data_iter:
        out = model(data)
        predictions.append(out.unsqueeze(0))
        all_labels.append(labels.unsqueeze(0))
    pred = torch.cat(predictions, axis=-1).squeeze().round().to('cpu')
    true = torch.cat(all_labels, axis=-1).squeeze().to('cpu')
    recalls = []
    precisions = []
    f1_scores = []
    for selected_class in range(2):
        tp = ((pred == selected_class) & (true == selected_class)).sum()
        fp = ((pred == selected_class) & (true != selected_class)).sum()
        fn = ((pred != selected_class) & (true == selected_class)).sum()
        recall = tp / (tp + fn) if tp + fn != 0 else 0
        precision = tp / (tp + fp) if tp + fp != 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if precision + recall != 0 else 0
        recalls.append(recall)
        precisions.append(precision)
        f1_scores.append(f1)
    return pred, true, recalls, precisions, f1_scores

def eval_on_test_set(model):
    _, _, recalls, precisions, f1_scores = eval_model(model, test_iter)
    print(f"""Run on Test Data:
    Down: Recall: {recalls[0]}\tPrecision: {precisions[0]}\tF1: {f1_scores[0]}
    Up: Recall: {recalls[1]}\tPrecision: {precisions[1]}\tF1: {f1_scores[1]}
    AVERAGE: Recall: {sum(recalls) / 2}\tPrecision: {sum(precisions) / 2}\tF1: {sum(f1_scores) / 2}""")

def eval_summary(epoch):
        _, _, recalls, precisions, f1_scores = eval_model(model, validation_iter)
        print(f"""Epoch {epoch} Validation:
Down: Recall: {recalls[0]}\tPrecision: {precisions[0]}\tF1: {f1_scores[0]}
Up: Recall: {recalls[1]}\tPrecision: {precisions[1]}\tF1: {f1_scores[1]}
AVERAGE: Recall: {sum(recalls) / 2}\tPrecision: {sum(precisions) / 2}\tF1: {sum(f1_scores) / 2}""")

## Training Loop

In [41]:
def train_model(model, data_iter, epochs, optimizer, scheduler, loss_func):
    epoch_average_losses = []
    with tqdm.notebook.trange(epochs, desc='training', unit='epoch') as epoch_iter:
        for epoch in epoch_iter:
            model.train()
            epoch_loss = 0
            epoch_samples = 0
            with tqdm.notebook.tqdm(data_iter, desc=f"epoch {epoch + 1}", unit='batch', total=len(data_iter)) as batch_iter:
                for i, (data, labels) in enumerate(batch_iter, start=1):
                    optimizer.zero_grad()
                    output = model(data)
                    loss = loss_func(output, labels.squeeze())
                    epoch_loss += loss.item()
                    epoch_samples += len(labels)
                    loss.backward()
                    optimizer.step()

                    batch_iter.set_postfix(mean_epoch_loss=epoch_loss / i)
                if epoch % 10 == 0:
                    eval_summary(epoch)





            avg_epoch_loss = epoch_loss / epoch_samples
            scheduler.step(avg_epoch_loss)
            epoch_average_losses.append(avg_epoch_loss)

    return epoch_average_losses

In [42]:
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
EPOCHS = 10

# Pre Training Stats

In [43]:
model = BagOfWords(len(vocab), 1000).to(device)
eval_on_test_set(model)
print('\n')
eval_summary(0)

Run on Test Data:
    Down: Recall: 1.0	Precision: 0.5098039507865906	F1: 0.6753246784210205
    Up: Recall: 0.0	Precision: 0	F1: 0
    AVERAGE: Recall: 0.5	Precision: 0.2549019753932953	F1: 0.33766233921051025


Epoch 0 Validation:
Down: Recall: 1.0	Precision: 0.71875	F1: 0.8363636136054993
Up: Recall: 0.0	Precision: 0	F1: 0
AVERAGE: Recall: 0.5	Precision: 0.359375	F1: 0.41818180680274963


# Train Model

In [44]:
loss_func = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, patience=3, threshold=0.01)
history = train_model(model, train_iter, EPOCHS, optimizer, scheduler, loss_func)

training:   0%|          | 0/10 [00:00<?, ?epoch/s]

epoch 1:   0%|          | 0/99 [00:00<?, ?batch/s]

Epoch 0 Validation:
Down: Recall: 0.0	Precision: 0	F1: 0
Up: Recall: 1.0	Precision: 0.28125	F1: 0.4390243887901306
AVERAGE: Recall: 0.5	Precision: 0.140625	F1: 0.2195121943950653


epoch 2:   0%|          | 0/99 [00:00<?, ?batch/s]

epoch 3:   0%|          | 0/99 [00:00<?, ?batch/s]

...

epoch 6:   0%|          | 0/99 [00:00<?, ?batch/s]

epoch 7:   0%|          | 0/99 [00:00<?, ?batch/s]

epoch 8:   0%|          | 0/99 [00:00<?, ?batch/s]

epoch 9:   0%|          | 0/99 [00:00<?, ?batch/s]

epoch 10:   0%|          | 0/99 [00:00<?, ?batch/s]

In [45]:
import os
if not os.path.isdir('../Saved_Models'):
    os.mkdir('../Saved_Models')
torch.save(model, '../Saved_Models/bag_of_words.pt')
bag = torch.load('../Saved_Models/bag_of_words.pt')

# Post Training Stats

In [46]:
eval_summary(EPOCHS)
print('\n')
eval_on_test_set(bag)

Epoch 10 Validation:
Down: Recall: 0.0	Precision: 0	F1: 0
Up: Recall: 1.0	Precision: 0.28125	F1: 0.4390243887901306
AVERAGE: Recall: 0.5	Precision: 0.140625	F1: 0.2195121943950653


Run on Test Data:
    Down: Recall: 0.0	Precision: 0	F1: 0
    Up: Recall: 1.0	Precision: 0.4901960790157318	F1: 0.6578947305679321
    AVERAGE: Recall: 0.5	Precision: 0.2450980395078659	F1: 0.32894736528396606


# Write results to CSV

In [47]:
def run_on_data(model, data_iter):
    model.eval()
    predictions, all_labels= [], []
    for (data, labels) in data_iter:
        out = model.predict_proba(data)
        predictions.append(out.unsqueeze(0))
        all_labels.append(labels.unsqueeze(0))
    pred = torch.cat(predictions, axis=-1).squeeze()
    true = torch.cat(all_labels, axis=-1).squeeze()
    return pred, true


In [48]:
train_pred, train_labels = run_on_data(model, train_iter)
valid_pred, valid_labels = run_on_data(model, validation_iter)
test_pred, test_labels = run_on_data(model, test_iter)

In [49]:
import pandas as pd
train_results = pd.DataFrame({"Labels": train_labels.to('cpu'), "Predictions": train_pred.to('cpu').detach().numpy()})
#, "Date": #train_data['Date']})
valid_results = pd.DataFrame({"Labels": valid_labels.to('cpu'), "Predictions": valid_pred.to('cpu').detach().numpy()})
#, "Date": validation_data['Date']})
test_results = pd.DataFrame({"Labels": test_labels.to('cpu'), "Predictions": test_pred.to('cpu').detach().numpy()})
#, "Date": test_data['Date']})

train_results.to_csv('../Results/train/BOW.csv', index=False)
valid_results.to_csv('../Results/validation/BOW.csv', index=False)
test_results.to_csv('../Results/test/BOW.csv', index=False)